# Creating a logistic regression to predict absenteeism

## Import relevant libraries

In [1]:
import pandas as pd
import numpy as np

## Import the data

In [2]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')
data_preprocessed

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,5,2,179,22,40,237.656,22,1,2,0,8
696,1,0,0,0,5,2,225,26,28,237.656,24,0,1,2,3
697,1,0,0,0,5,3,330,16,28,237.656,25,1,0,0,8
698,0,0,0,1,5,3,235,16,32,237.656,25,1,0,0,2


## create the target

In [3]:
# we will take the median value of the 'Absenteeism time in Hours' and use it as a cut-off line
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [4]:
# assign <=3, moderately absent (or 0), and >=4 excessively absent (0r 1)
targets = np.where(data_preprocessed['Absenteeism Time in Hours']>data_preprocessed['Absenteeism Time in Hours'].median(),1,0)

In [5]:
data_preprocessed['Excessive Absenteeism']=targets
data_preprocessed.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


## comment on the targets

In [6]:
# by using the median(), we implicitly balanced the dataset
# this prevent model from learning to output only 0's and 1's
targets.sum()/targets.shape[0]

# balance of 45:55 is sufficient

0.45571428571428574

## create the checkpoint

In [7]:
# with backward elimination, we drop 'day of the week', 'daily work load average', and 'distance to work'
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours','Day of the Week','Daily Work Load Average','Distance to Work'],axis=1)

## select input for the regression

In [8]:
data_with_targets.shape

(700, 12)

In [9]:
unscaled_inputs=data_with_targets.iloc[:,0:-1]

## standarise the data

In [10]:
## from sklearn.preprocessing import StandardScaler

## absenteeism_scaler = StandardScaler()

# This section is commented out since we will use custom scaler in order to 
# avoid the standarisatin of the dummy variables

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [12]:
unscaled_inputs.columns.values

array(['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [13]:
## omit the dummy variables
#columns_to_scale = ['Month Value','Day of the Week', 'Transportation Expense', 'Distance to Work','Age', 'Daily Work Load Average', 'Body Mass Index','Children', 'Pets']

# instead of choosing the columns to scale, we choose the columns to omit
columns_to_omit = ['Reason_1','Reason_2','Reason_3','Reason_4','Education']

In [14]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [15]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [16]:
## this line calculate and store the mean and the std and stored in absenteeism_scaler object
absenteeism_scaler.fit(unscaled_inputs)

C:\Users\yu298\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CustomScaler(columns=['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4',
                      'Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'],
             copy=None, with_mean=None, with_std=None)

In [17]:
## apply scaling mechanism
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)
scaled_inputs

,Reason 1,Reason 2,Reason 3,Reason 4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,-0.577350,-0.092981,-0.314485,0.821365,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,-0.577350,-0.092981,-0.314485,-1.217485,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,-0.577350,-0.092981,-0.314485,0.821365,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1.732051,-0.092981,-0.314485,-1.217485,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,-0.577350,-0.092981,-0.314485,0.821365,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1.732051,-0.092981,-0.314485,-1.217485,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1.732051,-0.092981,-0.314485,-1.217485,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1.732051,-0.092981,-0.314485,-1.217485,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,-0.577350,-0.092981,-0.314485,0.821365,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [18]:
scaled_inputs.shape

(700, 11)

## split the data into train& test and shuffle

### import the relevant module

In [19]:
from sklearn.model_selection import train_test_split

### split

In [20]:
## split the inputs into train and test variables
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state = 20)

## Logistic regression with sklearn

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

### training the model

In [22]:
reg = LogisticRegression()

In [23]:
reg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
## returns the mean accuracy on the given test data and labels
reg.score(x_train,y_train)

0.7875

## manually check the accuracy

In [25]:
model_outputs = reg.predict(x_train)
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [26]:
model_outputs == y_train

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,

In [27]:
## counting the number of entries that model_output == y_train
np.sum((model_outputs==y_train))

441

In [28]:
## total number of the entries
model_outputs.shape[0]

560

In [29]:
## calculate the accuracy
np.sum((model_outputs==y_train))/model_outputs.shape[0]

0.7875

## finding intercept and coefficients

In [30]:
## finding intercept
reg.intercept_

array([-0.17163628])

In [31]:
## finding coefficients
reg.coef_

array([[ 2.06866621,  0.33461133,  1.56034928,  1.31301781,  0.18521305,
         0.69049022, -0.19796143,  0.32782753, -0.31318895,  0.37213564,
        -0.32435396]])

In [32]:
## we want to know which coefficents refers to which variable
feature_name = unscaled_inputs.columns.values
feature_name

array(['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [33]:
# summarise feature names and their coefficients by creating the new dataframe
summary_table = pd.DataFrame(columns=['Feature name'],data=feature_name)

# add new column of coefficient 
summary_table['coefficient'] = np.transpose(reg.coef_)
summary_table

,Feature name,coefficient
0,Reason 1,2.068666
1,Reason 2,0.334611
2,Reason 3,1.560349
3,Reason 4,1.313018
4,Month Value,0.185213
5,Transportation Expense,0.690490
6,Age,-0.197961
7,Body Mass Index,0.327828
8,Education,-0.313189
9,Children,0.372136


In [34]:
## adding intercept to the table
summary_table.index = summary_table.index + 1
summary_table.loc[0]= ['Intercept',reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature name,coefficient
0,Intercept,-0.171636
1,Reason 1,2.068666
2,Reason 2,0.334611
3,Reason 3,1.560349
4,Reason 4,1.313018
5,Month Value,0.185213
6,Transportation Expense,0.690490
7,Age,-0.197961
8,Body Mass Index,0.327828
9,Education,-0.313189


## interpreting the coefficient

In [35]:
## finding odds by taking the exponential
summary_table['odd ratio'] = np.exp(summary_table.coefficient)

In [36]:
# sort the table in a descending order
summary_table.sort_values('odd ratio',ascending=False)

,Feature name,coefficient,odd ratio
1,Reason 1,2.068666,7.914260
3,Reason 3,1.560349,4.760484
4,Reason 4,1.313018,3.717375
6,Transportation Expense,0.690490,1.994693
10,Children,0.372136,1.450830
2,Reason 2,0.334611,1.397397
8,Body Mass Index,0.327828,1.387950
5,Month Value,0.185213,1.203475
0,Intercept,-0.171636,0.842285
7,Age,-0.197961,0.820401


In [37]:
## interpretation here:
# A feature is not particularly important if:
#   its coefficeint is around 0
#   its odd ratio is around 1
# For a unit change in the standarised feature, the odds increase by a 
# multiple equal to the odds ratio (1 = no change)

# daily work load average, distance to work, day of the week seem to not affect the model much

### standarising only the numerical variables

When the variables are standarised, so was all the dummy variables.  
However, this make it not possible to compare between the dummy variables.
e.g. On the table avobe, we could have said Reason 1 is 7.98 times more likely 
than reason 0 (which is the benchmark)
To solve this, we go back to the step where we standarised the variables.

## Backword elimination

The idea is that we can simplify our model by removing all features which have close to no contribution to the model.

Which we have the p-values, we get rid of all coeffieints with p-values > 0.05

Although p-value is not included in sklearn ,if the weight is small enough, it wont make a difference anyway.

If we remove these variables, the rest of our model should not really change in terms of coefficient values.

## Testing the model

In [38]:
## check the accuracy of our model using 'test' data this time
reg.score(x_test,y_test)

# accuracy should be smaller than the trained data accuracy.  
# if the drop is too big then the model probably over-fitted.

0.7357142857142858

In [39]:
## instead of 0 and 1, we get a probability of being 0 and 1
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.70823956, 0.29176044],
       [0.57293437, 0.42706563],
       [0.39748224, 0.60251776],
       [0.78749486, 0.21250514],
       [0.06719693, 0.93280307],
       [0.31207365, 0.68792635],
       [0.28664053, 0.71335947],
       [0.08120082, 0.91879918],
       [0.80017608, 0.19982392],
       [0.74995457, 0.25004543],
       [0.46819712, 0.53180288],
       [0.18512014, 0.81487986],
       [0.04121567, 0.95878433],
       [0.75142701, 0.24857299],
       [0.23903853, 0.76096147],
       [0.53714554, 0.46285446],
       [0.53420084, 0.46579916],
       [0.52102189, 0.47897811],
       [0.40678406, 0.59321594],
       [0.02773414, 0.97226586],
       [0.70236673, 0.29763327],
       [0.78749486, 0.21250514],
       [0.40670612, 0.59329388],
       [0.40670612, 0.59329388],
       [0.17559234, 0.82440766],
       [0.75454373, 0.24545627],
       [0.48937739, 0.51062261],
       [0.87896092, 0.12103908],
       [0.13142638, 0.86857362],
       [0.78749486, 0.21250514],
       [0.

In [40]:
## the first column is the probability of observation being 0, the seoncd is being 1
predicted_proba[:,1]

array([0.29176044, 0.42706563, 0.60251776, 0.21250514, 0.93280307,
       0.68792635, 0.71335947, 0.91879918, 0.19982392, 0.25004543,
       0.53180288, 0.81487986, 0.95878433, 0.24857299, 0.76096147,
       0.46285446, 0.46579916, 0.47897811, 0.59321594, 0.97226586,
       0.29763327, 0.21250514, 0.59329388, 0.59329388, 0.82440766,
       0.24545627, 0.51062261, 0.12103908, 0.86857362, 0.21250514,
       0.37628326, 0.69395995, 0.71016857, 0.55802337, 0.21250514,
       0.56289228, 0.2084135 , 0.80857331, 0.41359763, 0.62953752,
       0.20379034, 0.4242538 , 0.22639945, 0.10462937, 0.85742011,
       0.6523714 , 0.70507958, 0.29176044, 0.21052579, 0.19534428,
       0.56865035, 0.07901966, 0.68792635, 0.2680619 , 0.8589383 ,
       0.45311982, 0.92991324, 0.21727659, 0.08573649, 0.08997337,
       0.70893813, 0.67646259, 0.28670381, 0.85947072, 0.18954376,
       0.27039693, 0.01391914, 0.2084135 , 0.81053586, 0.29013739,
       0.2084135 , 0.06812914, 0.92885154, 0.47883471, 0.63880

## Save the model

In [41]:
import pickle

In [43]:
# pickel has the form
# with open ('file name', write bytes) as file:
#    pickle.dump('object to be dumped', file)
# dump = 'save'
with open ('model', 'wb') as file:
    pickle.dump(reg,file)

In [44]:
# we must also save the absenteeism_scaler too
# As this was used to standarised the variables, and this info will be used 
# for preprocessing in the future
with open ('scaler', 'wb') as file:
    pickle.dump(absenteeism_scaler,file)